# BTC Trading Data

Cryptocurrencies have enterend the public psyche and have shown that they can even attract conventional investors. Bitcoin has seen a huge jump in corporate investors. While many platforms that sell cryptocurrencies give minute by minute data points, it is far from a trading strategy and lacks various metrics that traditional investors use. 

This project is a proof of concept for visualizing such metrics and calculations using about 5 years of Bitcoin data. 

In [1]:
#import required packages

import requests
import os
import numpy as np
import pandas as pd
import pandas_profiling

import plotly.graph_objects as go
from plotly.offline import init_notebook_mode
from plotly.subplots import make_subplots
init_notebook_mode(connected = True)

from datetime import datetime

import pickle
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# setup API for cryptocompare

apiKey = os.environ.get('API_KEY')


In [3]:
from_symbol = 'BTC'
to_symbol = 'USD'
exchange = 'Bitstamp'
datetime_interval = 'day'

In [4]:
def get_filename(from_symbol, to_symbol, exchange, datetime_interval, download_date):
    return '%s_%s_%s_%s_%s.csv' % (from_symbol, to_symbol, exchange, datetime_interval, download_date)


def download_data(from_symbol, to_symbol, exchange, datetime_interval):
    supported_intervals = {'minute', 'hour', 'day'}
    assert datetime_interval in supported_intervals,\
        'datetime_interval should be one of %s' % supported_intervals

    print('Downloading %s trading data for %s %s from %s' %
          (datetime_interval, from_symbol, to_symbol, exchange))
    base_url = 'https://min-api.cryptocompare.com/data/histo'
    url = '%s%s' % (base_url, datetime_interval)

    params = {'fsym': from_symbol, 'tsym': to_symbol,
              'limit': 2000, 'aggregate': 1,
              'e': exchange}
    request = requests.get(url, params=params)
    data = request.json()
    return data


def convert_to_dataframe(data):
    df = pd.json_normalize(data, ['Data'])
    df['datetime'] = pd.to_datetime(df.time, unit='s')
    df = df[['datetime', 'low', 'high', 'open',
             'close', 'volumefrom', 'volumeto']]
    return df


def filter_empty_datapoints(df):
    indices = df[df.sum(axis=1) == 0].index
    print('Filtering %d empty datapoints' % indices.shape[0])
    df = df.drop(indices)
    return df


data = download_data(from_symbol, to_symbol, exchange, datetime_interval)
df = convert_to_dataframe(data)
df = filter_empty_datapoints(df)

current_datetime = datetime.now().date().isoformat()
filename = get_filename(from_symbol, to_symbol, exchange, datetime_interval, current_datetime)
print('Saving data to %s' % filename)
df.to_csv(filename, index=False)

Filtering 0 empty datapoints
Saving data to BTC_USD_Bitstamp_day_2021-05-20.csv


# Read the data

In [5]:
def read_dataset(filename):
    print('Reading data from %s' % filename)
    df = pd.read_csv(filename)
    df.datetime = pd.to_datetime(df.datetime) # change type from object to datetime
    df = df.set_index('datetime') 
    df = df.sort_index() # sort by datetime
    print(df.shape)
    return df

df = read_dataset(filename)

Reading data from BTC_USD_Bitstamp_day_2021-05-20.csv
(2001, 6)


In [6]:
df.head()

,low,high,open,close,volumefrom,volumeto
datetime,,,,,,
2015-11-28,350.41,359.44,357.14,356.39,6228.08,2208751.82
2015-11-29,354.45,373.15,356.78,371.04,7667.10,2798580.83
2015-11-30,367.11,383.00,371.48,377.98,17776.96,6688039.27
2015-12-01,354.56,379.00,376.70,362.95,15389.67,5610058.11
2015-12-02,348.64,363.06,363.06,359.97,12043.46,4282488.25


In [7]:
df.tail()

,low,high,open,close,volumefrom,volumeto
datetime,,,,,,
2021-05-16,43769.40,49800.00,46708.89,46444.75,5750.08,2.675827e+08
2021-05-17,42100.00,46648.50,46444.75,43593.39,12262.72,5.400039e+08
2021-05-18,42023.69,45872.05,43593.39,42877.75,6275.91,2.764313e+08
2021-05-19,30066.00,43601.63,42877.75,36780.43,32468.86,1.233364e+09
2021-05-20,35000.00,43000.00,36780.43,39974.11,12677.03,5.039339e+08


# Feature Engineering for Candlestick Chart via Plotly

In [8]:
# set color for candlestick chart

INCREASING_COLOR = '#17BECF'
DECREASING_COLOR = '#7F7F7F'

In [9]:
# set initial candlestick options

fig = go.Figure(data=[go.Candlestick(
    x = df.index,
    open = df['open'],
    high = df['high'],
    low = df['low'],
    close = df['close'],
    yaxis='y',
    name = 'Candlestick')
                     ]);

In [10]:
# add range buttons

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            x = 0, y = 0.9,
            bgcolor = 'rgba(150, 200, 250, 0.4)',
            font = dict( size = 13 ),
            buttons=list([
                dict(count=1,
                     label='reset',
                     step='all'),
                dict(count=1,
                     label='1yr',
                     step='year',
                     stepmode='backward'),
                dict(count=3,
                     label='3 mo',
                     step='month',
                     stepmode='backward'),
                dict(count=1,
                     label='1 mo',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type='date'
    )
);

In [11]:
# add moving average

def movingaverage(interval, window_size=10):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

In [12]:
mv_y = movingaverage(df.close)
mv_x = list(df.index)

# clip ends
mv_y = mv_y[5:-5]
mv_x = mv_x[5:-5]


fig.add_trace(go.Scatter(
    x=mv_x,
    y=mv_y,
    mode = 'lines',
    line_color = 'black',
    name = 'Moving Average')
             );


In [13]:
# set volume bar chart colors

colors = []

for i in range(len(df.close)):
    if i !=0:
        if df.close[1] > df.close[i-1]:
            colors.append(INCREASING_COLOR)
        else:
            colors.append(DECREASING_COLOR)
    else:
        colors.append(DECREASING_COLOR)

In [14]:
# add bolinger bands

def bbands(price, window_size=10, num_of_std=5):
    rolling_mean = price.rolling(window = window_size).mean()
    rolling_std = price.rolling(window = window_size).std()
    upper_band = rolling_mean + (rolling_std * num_of_std)
    lower_band = rolling_mean - (rolling_std * num_of_std)
    return rolling_mean, upper_band, lower_band

bb_avg, bb_upper, bb_lower = bbands(df.close)


# bolinger band upper bound
fig.add_trace(go.Scatter(x=df.index, y=bb_upper, yaxis='y',
                         line = dict(width=1),
                         line_color = 'gray', hoverinfo='none',
                         legendgroup='Bollinger Bands', name='Bollinger Bands')
             );

# bolinger band lower bound
fig.add_trace(go.Scatter(x=df.index, y=bb_lower, yaxis='y',
                         line = dict(width=1),
                         line_color = 'gray', hoverinfo='none',
                         legendgroup='Bollinger Bands', showlegend=False)
             );

# simple rolling mean 
fig.add_trace(go.Scatter(x=df.index, y=bb_avg, yaxis='y',
                         line = dict(width=1),
                         line_color = 'black', hoverinfo='none', legendgroup='Bollinger Bands',
                         showlegend=False));


In [15]:
# plot

fig.show();